In [26]:
import tweepy #api package handler
import csv #to store results in csv files 
import os 
import json
import datetime
import time
from collections import Counter
import operator

In [27]:
#credentials
api_key = 'BpiMXpXrKoyq6TRaVEvu4NUUr'
api_key_secret = 'FdzzCKpQ4s75cGKXKQQCnSZNvu3nbsqlsX0vlSMJ7wPk9vLRxz'

In [28]:
#get access to tweeter api
auth = tweepy.AppAuthHandler(api_key, api_key_secret) #send OAuth request
api = tweepy.API(auth) #create API object

In [29]:
#get keywords from user
#input must be: word1, word2, .....
print('Enter keywords as folling: word1, word2, ....')
userInput = input() #ask user for inputs
userInput = userInput.replace(', ', ',').replace(' ,', ',') #prepare input to convert it into array
keywordsArray = userInput.split(',') #conver input to array
#print(keywordsArray)

Enter keywords as folling: word1, word2, ....


 Doctor, Japan


In [32]:
#consume api and store result in array of 5 columns [key, created_at, text, username, trend]

total_found = 0 #to count how many rows received from api
start_time = time.time() #start chronometer

tweetArray = [] #toring api result in array
trendCollection = [] #collect all posts trends

for key in keywordsArray:
    for tweet in tweepy.Cursor(api.search, q=key).items(200): #nb of queries set to 200 to not overquery the api, to set queries to unlimited just put: .items()
        
        isEmpty = len(tweet.entities['hashtags']) #check if post has hashtag
        trend = tweet.entities['hashtags'][0]['text'] if isEmpty!=0 else ''    #get first tag only
 
        created_at = tweet.created_at
        text = tweet.text
        username = tweet.user.screen_name

        tweetArray.append([key, created_at, text, username, trend]) #store tweet record
        trendCollection.append(trend) #store tweet tag

        total_found = total_found + 1 #track records got as an api resonse
        
end_time = time.time() #chrono ends

In [36]:
#### Prepare Tags array helpers, ######
tags = list(set(trendCollection)) #delete duplication from trendCollection array
tagWithOccurance = [] #array of tag with its occurence

for tag in tags:
    tagWithOccurance.append([tag, trendCollection.count(tag)])
#print(tags)
#print(tagWithOccurance)

In [37]:
##########################################################

In [38]:
######## GLOBAL API RESULT & CREATE CSV ##################

In [39]:
###Default tweets :: array of 6 columns with a tagOccurence [key, created_at, text, username, trend, tagOccurence]
finalTweet = []
for item in tweetArray:
    tagName = item[4] #get row tagname
    tagIndex = tags.index(tagName) #get index of the tagName on tag[]
    tagOccurence = tagWithOccurance[tagIndex][1] #get occurence in tagOccurence0-
    finalTweet.append([item[0], item[1], item[2], item[3], item[4], tagOccurence]) #store record 

In [40]:
##Default tweets csv
with open('defaultResults.csv', 'w', newline='', encoding='utf-8') as file:
    filednames = ["keyword", "created_at", "text", "username", "tag", "tag occurence"] #define csv header
    writer = csv.DictWriter(file, fieldnames=filednames) 
    writer.writeheader() #write csv's header
    writer = csv.writer(file, delimiter=',')
    writer.writerows(finalTweet) #store all finalTweet's record (default records)
    file.close()

In [41]:
###Sorted tweets by tag occurence
finalTweet.sort(key=lambda x: x[5], reverse=True) #sorted on descandant

In [42]:
##Sorted tweets by tag occurence csv
with open('classifiedByOccurence.csv', 'w', newline='', encoding='utf-8') as file:
    filednames = ["keyword", "created_at", "text", "username", "tag", "tag occurence"] #define csv header
    writer = csv.DictWriter(file, fieldnames=filednames)
    writer.writeheader() #write csv's header
    writer = csv.writer(file, delimiter=',')
    writer.writerows(finalTweet) #store all finalTweet's record (default records)
    file.close()   

In [43]:
######################################

In [44]:
######## TOP 10 TAGS ################

In [45]:
#----- FUNCTIONS --------#

In [46]:
#Day Top tag, returns list of top hashtags of a day of the week
def getTopDayTag(nbDays):
    tagDayCollection_temp = [] #colloect tags of the day
    
    askedDay = datetime.date.today() - datetime.timedelta(days=nbDays) #find the day's date
    dayStart = datetime.datetime(askedDay.year, askedDay.month, askedDay.day, 0) #set day start
    dayEnd = datetime.datetime(askedDay.year, askedDay.month, askedDay.day, 23, 59, 59) #set day end
    
    for item in finalTweet:
        if item[1] >= dayStart and item[1] <= dayEnd:
            tagDayCollection_temp.append(item[4]) #store item's hashtag
    
    topDayTag = sorted(set(tagDayCollection_temp), key = lambda ele: tagDayCollection_temp.count(ele), reverse=True) #sort resulted hashtags by occurence descendant
    if len(topDayTag) != 0: #if topDayTag was not empty
        topDayTag.remove('')  #to remove the no tag case
    
    return topDayTag

In [47]:
#Hour Top Tag, returns list of top hashtags of a given hour of the day
def getTopHourTag(nbHours):
    tagHourCollection_temp = [] #colloect tags of this hour

    askedHour = datetime.datetime.today() - datetime.timedelta(hours=nbHours) #find the current hour
    hourStart = datetime.datetime(askedHour.year, askedHour.month, askedHour.day, askedHour.hour, 0) #set hour start
    hourEnd = datetime.datetime(askedHour.year, askedHour.month, askedHour.day, askedHour.hour, 59) #set hour end

    for item in finalTweet:
        if item[1] >= hourStart and item[1] <= hourEnd: #if datStart < item's date < dayEnd
            tagHourCollection_temp.append(item[4]) #store item's hashtag

    #hour tags
    topHourTag = sorted(set(tagHourCollection_temp), key = lambda ele: tagHourCollection_temp.count(ele), reverse=True) #sort resulted hashtags by occurence descendant
    if len(topHourTag) != 0: #if topDayTag was not empty
        topHourTag.remove('') #to remove the no tag case

    return topHourTag

In [48]:
#----- DISPLAY RESULT --------#

In [49]:
#array to string
def listToString(L):  
    return ", ".join(str(x) for x in L)

In [50]:
#General Details
time_lapsed = format(end_time - start_time, ".4f")
print('\033[4m' + 'Total time spent to get results' + '\033[0m' + ':', end =" ")
print(time_lapsed + ' seconds') #chorometer counts
print('\033[4m' + 'Total rows found from api' + '\033[0m' + ':', end =" ")
print(total_found, end=" ") #total rows caught from api
print('rows')

Total time spent to get results: 25.8001 seconds
Total rows found from api: 400 rows


In [52]:
#Top10 tag week tag
print('Top 10 Hashtags of the week : ')
topWeekTag = sorted(set(trendCollection), key = lambda ele: trendCollection.count(ele), reverse=True) #sort hashtags by occurence, descendant
topWeekTag.remove('')  #to remove the no hashtag case
print(str(topWeekTag[:10]).strip('[]')) #print as string list


Top 10 Hashtags of the week : 
'SWITCH_ON', 'tbs', 'TWICE', 'MiVidaMiOxigeno', '三浦春馬', 'BARFOUT', 'Metroid', 'Purgatell', 'YUTA', 'Hybrid'


In [54]:
#Top10 tag days of the week tag
print('Top 10 Hashtags for each day of the week : ')
for i in range(7):
    askedDay = datetime.date.today() - datetime.timedelta(days=i) #set a day of the 7 days (1 week)
    print(askedDay, end=" ")
    print(':', end=" ")
    result = listToString(getTopDayTag(i)[:10]) #get top 10 hashtags of setted day
    print(result) 

Top 10 Hashtags for each day of the week : 
2021-07-21 : 
2021-07-20 : SWITCH_ON, tbs, TWICE, MiVidaMiOxigeno, 三浦春馬, BARFOUT, Metroid, Purgatell, YUTA, Hybrid
2021-07-19 : 
2021-07-18 : 
2021-07-17 : 
2021-07-16 : 
2021-07-15 : 


In [55]:
#Top10 tags per hours of tag
print('Top 10 Hashtags for each hour of today')
print(datetime.date.today())
for i in range(24):
    askedHour = datetime.datetime.today() - datetime.timedelta(hours=i) #set an hour of the current day
    print(askedHour.hour, end=" ")
    print('\t:', end=" ")
    result = listToString(getTopHourTag(i)[:10]) #get top 10 hashtags of setted hour
    print(result)

Top 10 Hashtags for each hour of today
2021-07-21
0 	: 
23 	: SWITCH_ON, tbs, TWICE, MiVidaMiOxigeno, 三浦春馬, BARFOUT, Metroid, Purgatell, YUTA, Hybrid
22 	: 
21 	: 
20 	: 
19 	: 
18 	: 
17 	: 
16 	: 
15 	: 
14 	: 
13 	: 
12 	: 
11 	: 
10 	: 
9 	: 
8 	: 
7 	: 
6 	: 
5 	: 
4 	: 
3 	: 
2 	: 
1 	: 
